# ConIndVar

- Concat Independent Variable
- 독립변수 합치기

- 대상 변수: 사용할 변수 목록.xlsx 참고


해야할 일

1. 결측값 대치
    1. 안 하기! XGBoost는 결측값에 강건하다
    2. missingpy 라이브러리의 missforest 사용
    3. xgbimputer 라이브러리 쓰기!
        - 이 방법 쓰려면 좀 복잡할듯
    4. 원래 하던대로 하기(최빈값, 평균값 등등)

2. 이상치 제거...?
    1. 한다
    2. 안 한다

3. 데이터 가공하기. 고혈압 여부 등





In [65]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc

# 기본 글꼴을 D2Coding ligature으로 변경
rc("font", family="D2Coding ligature")

In [66]:
print(plt.rcParams['font.family'])

['D2Coding ligature']


- 원본 데이터 불러오기

- GH_9622 데이터셋 기준으로 인덱스 조정해야함.

In [67]:
AS1_01_EXAMINEE = pd.read_csv("./Dataset/AS1_01_EXAMINEE.csv", index_col=0, encoding='utf-8', low_memory=False)
AS1_02_GEN = pd.read_csv("./Dataset/AS1_02_GEN.csv", index_col=0, encoding='utf-8', low_memory=False)
AS1_03_DRSM = pd.read_csv("./Dataset/AS1_03_DRSM.csv", index_col=0, encoding='utf-8', low_memory=False)
AS1_04_ACTIVE = pd.read_csv("./Dataset/AS1_04_ACTIVE.csv", index_col=0, encoding='utf-8', low_memory=False)
AS1_05_MEDIC = pd.read_csv("./Dataset/AS1_05_MEDIC.csv", index_col=0, encoding='utf-8', low_memory=False)
AS1_06_DISEASE = pd.read_csv("./Dataset/AS1_06_DISEASE.csv", index_col=0, encoding='utf-8', low_memory=False)
AS1_07_TREAT = pd.read_csv("./Dataset/AS1_07_TREAT.csv", index_col=0, encoding='utf-8', low_memory=False)
AS1_08_DRUG = pd.read_csv("./Dataset/AS1_08_DRUG.csv", index_col=0, encoding='utf-8', low_memory=False)
AS1_13_SLEEP = pd.read_csv("./Dataset/AS1_13_SLEEP.csv", index_col=0, encoding='utf-8', low_memory=False)
AS1_16_DIET = pd.read_csv("./Dataset/AS1_16_DIET.csv", index_col=0, encoding='utf-8', low_memory=False)
AS1_17_FFQWEIGHT = pd.read_csv("./Dataset/AS1_17_FFQWEIGHT.csv", index_col=0, encoding='utf-8', low_memory=False)
AS1_18_BIOCHEM = pd.read_csv("./Dataset/AS1_18_BIOCHEM.csv", index_col=0, encoding='utf-8', low_memory=False)
AS1_19_ANTHRO = pd.read_csv("./Dataset/AS1_19_ANTHRO.csv", index_col=0, encoding='utf-8', low_memory=False)
GH = pd.read_csv("./Dataset/GH_9622.csv", index_col=0, encoding='utf-8', low_memory=False) # 경훈이가 가공해준 식이패턴 데이터             


In [68]:
print(len(AS1_01_EXAMINEE), len(GH))

10030 9622


In [69]:
mask = AS1_01_EXAMINEE.index.isin(GH.index)

AS1_01_EXAMINEE = AS1_01_EXAMINEE[mask]
AS1_02_GEN = AS1_02_GEN[mask]
AS1_03_DRSM = AS1_03_DRSM[mask]
AS1_04_ACTIVE = AS1_04_ACTIVE[mask]
AS1_05_MEDIC = AS1_05_MEDIC[mask]
AS1_06_DISEASE = AS1_06_DISEASE[mask]
AS1_07_TREAT = AS1_07_TREAT[mask]
AS1_08_DRUG = AS1_08_DRUG[mask]
AS1_13_SLEEP = AS1_13_SLEEP[mask]
AS1_16_DIET = AS1_16_DIET[mask]
AS1_17_FFQWEIGHT = AS1_17_FFQWEIGHT[mask]
AS1_18_BIOCHEM = AS1_18_BIOCHEM[mask]
AS1_19_ANTHRO = AS1_19_ANTHRO[mask]

In [70]:
VAR_TO_USE = pd.read_csv("./Docs/var_to_use_2024-04-18-1730I.csv")
VAR_TO_USE

,binary,cnt,cath0,cath1
0,AS1_SEX,AS1_AGE,AS1_JOBB,AS1_EDUA
1,AS1_TIED,AS1_MARRYA,AS1_DRINK,AS1_SMOKEA
2,HAS_HYPERTENSION,AS1_INCOME,NaN,AS1_PHYACTL
3,HAS_DIABETES,AS1_TOTALC,NaN,AS1_PHYACTM
4,HAS_HYPERLIPIDEMI,AS1_HVSMAM,NaN,AS1_PHYACTH
5,AS1_SLPAMSF,AS1_RGMEALFQA,NaN,AS1_HEALTH
6,NaN,HOMA_IR,NaN,NaN
7,NaN,WAIST_AVG,NaN,NaN
8,NaN,BMI,NaN,NaN
9,NaN,KHEI90,NaN,NaN


In [71]:
VAR_DF = pd.read_csv("./Docs/var_list.csv")
VAR_DF

,테이블명,변수명,변수명 변환,변수 유형,사용여부,가공여부
0,AS1_01_EXAMINEE,AS1_Sex,AS1_SEX,binary,NaN,NaN
1,AS1_01_EXAMINEE,AS1_Age,AS1_AGE,cnt,NaN,NaN
2,AS1_02_GEN,AS1_MarryA,AS1_MARRYA,cnt,NaN,NaN
3,AS1_02_GEN,AS1_JobB,AS1_JOBB,cath0,NaN,NaN
4,AS1_02_GEN,AS1_EduA,AS1_EDUA,cath1,NaN,NaN
5,AS1_02_GEN,AS1_Income,AS1_INCOME,cnt,NaN,NaN
6,AS1_03_DRSM,AS1_Drink,AS1_DRINK,cath0,NaN,NaN
7,AS1_03_DRSM,AS1_TotAlc,AS1_TOTALC,cnt,NaN,NaN
8,AS1_03_DRSM,AS1_SmokeA,AS1_SMOKEA,cath1,NaN,NaN
9,AS1_03_DRSM,AS1_HvSmAm,AS1_HVSMAM,cnt,NaN,NaN


## Data Engineering

- 결측값
- 새로운 변수 가공
- 스케일링?

In [72]:
result = pd.DataFrame(index=AS1_01_EXAMINEE.index)

### Imputation

해야할 일

- 77777 -> 0 또는 특정 값
- 99999 -> NaN

In [73]:
AS1_01_EXAMINEE.replace(99999, np.nan, inplace=True)
AS1_02_GEN.replace(99999, np.nan, inplace=True)
AS1_03_DRSM.replace(99999, np.nan, inplace=True)
AS1_04_ACTIVE.replace(99999, np.nan, inplace=True)
AS1_05_MEDIC.replace(99999, np.nan, inplace=True)
AS1_06_DISEASE.replace(99999, np.nan, inplace=True)
AS1_07_TREAT.replace(99999, np.nan, inplace=True)
AS1_08_DRUG.replace(99999, np.nan, inplace=True)
AS1_13_SLEEP.replace(99999, np.nan, inplace=True)
AS1_16_DIET.replace(99999, np.nan, inplace=True)
AS1_17_FFQWEIGHT.replace(99999, np.nan, inplace=True)
AS1_18_BIOCHEM.replace(99999, np.nan, inplace=True)
AS1_19_ANTHRO.replace(99999, np.nan, inplace=True)

In [74]:
print(f"""
{AS1_01_EXAMINEE.isna().sum()}
{AS1_02_GEN.isna().sum()}
{AS1_03_DRSM.isna().sum()}
{AS1_04_ACTIVE.isna().sum()}
{AS1_05_MEDIC.isna().sum()}
{AS1_06_DISEASE.isna().sum()}
{AS1_07_TREAT.isna().sum()}
{AS1_08_DRUG.isna().sum()}
{AS1_13_SLEEP.isna().sum()}
{AS1_16_DIET.isna().sum()}
{AS1_17_FFQWEIGHT.isna().sum()}
{AS1_18_BIOCHEM.isna().sum()}
{AS1_19_ANTHRO.isna().sum()}
""")



AS1_AREA      0
AS1_EDATE1    0
AS1_SEX       0
AS1_AGE       0
dtype: int64
AS1_FAMNUM         9
AS1_HAND          14
AS1_SEAS          74
AS1_MARRYA        38
AS1_MARRYAETC      0
AS1_RELIG         23
AS1_RELIGETC       0
AS1_RELIGPRT      85
AS1_JOBB          40
AS1_EDUA          53
AS1_RSDYR         35
AS1_HOUSETY       13
AS1_HOUSEPS       48
AS1_INCOME       133
dtype: int64
AS1_DRINK          37
AS1_DRDUA         155
AS1_DRQMO         142
AS1_TAK           104
AS1_TAKFQA        122
AS1_TAKAMA        121
AS1_TAKGS         191
AS1_BEER          104
AS1_BEERFQA       129
AS1_BEERAMA       136
AS1_BEERGS        270
AS1_RICE          104
AS1_RICEFQA       105
AS1_RICEAMA       110
AS1_RICEGS        121
AS1_WINE          104
AS1_WINEFQA       114
AS1_WINEAMA       114
AS1_WINEGS        146
AS1_SOJU          104
AS1_SOJUFQA       155
AS1_SOJUAMA       176
AS1_SOJUGS        487
AS1_HLIQ          104
AS1_HLIQFQA       110
AS1_HLIQAMA       113
AS1_HLIQGS        129
AS1_TOTALC        223

In [75]:
# 딕셔너리 초기화
result_dict = {}

# 테이블명을 키로, 변수명 변환을 값으로 딕셔너리 생성
for i, row in VAR_DF.iterrows():
   table_name = row['테이블명']
   var_trans = row['변수명 변환']
   if table_name not in result_dict:
       result_dict[table_name] = [var_trans]
   else:
       result_dict[table_name].append(var_trans)

print(result_dict)

{'AS1_01_EXAMINEE': ['AS1_SEX', 'AS1_AGE'], 'AS1_02_GEN': ['AS1_MARRYA', 'AS1_JOBB', 'AS1_EDUA', 'AS1_INCOME'], 'AS1_03_DRSM': ['AS1_DRINK', 'AS1_TOTALC', 'AS1_SMOKEA', 'AS1_HVSMAM'], 'AS1_04_ACTIVE': ['AS1_PHYACTL', 'AS1_PHYACTM', 'AS1_PHYACTH'], 'AS1_05_MEDIC': ['AS1_HEALTH', 'AS1_TIED'], 'AS1_06_DISEASE': ['HAS_HYPERTENSION', 'HAS_DIABETES', 'HAS_HYPERLIPIDEMI'], 'AS1_13_SLEEP': ['AS1_SLPAMSF'], 'AS1_16_DIET': ['AS1_RGMEALFQA'], 'AS1_18_BIOCHEM': ['HOMA_IR'], 'AS1_19_ANTHRO': ['WAIST_AVG', 'BMI'], '경훈': ['KHEI90', 'AMED', 'AMED_NONALC', 'DASH', 'PRAL', 'NEAP', 'DII']}


In [76]:
result = pd.concat([AS1_01_EXAMINEE[["AS1_SEX", "AS1_AGE"]],
                    AS1_02_GEN[["AS1_MARRYA", "AS1_JOBB", "AS1_EDUA", "AS1_INCOME"]],
                    AS1_03_DRSM[['AS1_DRINK', 'AS1_SMOKEA', 'AS1_HVSMAM']], # 'AS1_TOTALC'를 GH에서 불러옴
                    AS1_04_ACTIVE[['AS1_PHYACTL', 'AS1_PHYACTM', 'AS1_PHYACTH']],
                    AS1_05_MEDIC[['AS1_HEALTH', 'AS1_TIED']],
                    AS1_13_SLEEP[['AS1_SLPAMSF']],
                    AS1_16_DIET[['AS1_RGMEALFQA']],
                    GH[['AS1_TOTALC', 'KHEI90', 'AMED', 'AMED_NONALC', 'DASH', 'PRAL', 'NEAP', 'DII']]],
                    axis=1)

In [77]:
result['AS1_HVSMAM'].replace(77777, 0, inplace=True)

C:\Users\sh22h\AppData\Local\Temp\ipykernel_38392\2240191053.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  result['AS1_HVSMAM'].replace(77777, 0, inplace=True)


In [78]:
result.head()

,AS1_SEX,AS1_AGE,AS1_MARRYA,AS1_JOBB,AS1_EDUA,AS1_INCOME,AS1_DRINK,AS1_SMOKEA,AS1_HVSMAM,AS1_PHYACTL,...,AS1_SLPAMSF,AS1_RGMEALFQA,AS1_TOTALC,KHEI90,AMED,AMED_NONALC,DASH,PRAL,NEAP,DII
DIST_ID,,,,,,,,,,,,,,,,,,,,,
NIH23B1298125,1,53,2.0,2.0,3.0,5.0,3.0,1.0,0.0,1.0,...,2.0,3.0,23.15,61,6,5,30,125.648516,25.068314,2.062933
NIH23B1159376,2,44,2.0,5.0,1.0,2.0,1.0,0.0,0.0,7.0,...,2.0,2.0,0.00,35,3,3,24,119.929227,20.677190,2.699810
NIH23B1785393,1,47,2.0,3.0,1.0,2.0,3.0,3.0,8.0,2.0,...,1.0,3.0,23.57,38,3,2,15,141.011119,42.987319,1.297841
NIH23B1463054,2,43,2.0,3.0,1.0,2.0,3.0,0.0,0.0,2.0,...,1.0,3.0,0.29,51,2,2,21,141.744284,43.921116,1.274731
NIH23B1751168,1,61,2.0,3.0,3.0,1.0,3.0,3.0,20.0,0.0,...,2.0,3.0,43.41,27,4,4,19,127.987263,28.310385,1.615857


### Data 가공하기

- 고혈압여부: HAS_HYPERTENSION
- 당뇨여부: HAS_DIABETES
- 고지혈증: HAS_HYPERLIPIDEMI
- LDL: LDL
- H0MA_IR: H0MA_IR
- 허리둘레: WAIST_AVG
- BMI: BMI


In [79]:
processed_df = pd.DataFrame(index=AS1_01_EXAMINEE.index)
processed_df.head()

""
DIST_ID
NIH23B1298125
NIH23B1159376
NIH23B1785393
NIH23B1463054
NIH23B1751168


#### 고혈압여부: HAS_HYPERTENSION
> pdht == 2 or drughtcu == 2 or 3회평균_sys >= 140 or 3회평균_dia >=90

In [80]:
pdht = AS1_06_DISEASE[["AS1_PDHT"]]
drughtcu = AS1_08_DRUG[['AS1_DRUGHTCU']]
sys = AS1_19_ANTHRO[['AS1_BPLIE1S', 'AS1_BPLIE2S', 'AS1_BPLIE3S']].mean(axis=1)
dia = AS1_19_ANTHRO[['AS1_BPLIE1D', 'AS1_BPLIE2D', 'AS1_BPLIE3D']].mean(axis=1)

df = pd.concat([pdht, drughtcu], axis=1)
df["AS1_AVG_SYS"] = sys
df["AS1_AVG_DIA"] = dia
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9622 entries, NIH23B1298125 to NIH23B1160138
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   AS1_PDHT      9621 non-null   float64
 1   AS1_DRUGHTCU  9115 non-null   float64
 2   AS1_AVG_SYS   9612 non-null   float64
 3   AS1_AVG_DIA   9612 non-null   float64
dtypes: float64(4)
memory usage: 375.9+ KB


In [81]:
df["AS1_DRUGHTCU"].value_counts()

AS1_DRUGHTCU
77777.0    8003
2.0        1043
1.0          69
Name: count, dtype: int64

In [82]:
conditions = (
    (df['AS1_PDHT'] == 2) | 
    (df['AS1_DRUGHTCU'] == 2) | 
    (df['AS1_AVG_SYS'] >= 140) | 
    (df['AS1_AVG_DIA'] >= 90)
)

# df['HAS_HYPERTENSION'] = np.where(conditions, 1, 0)
processed_df['HAS_HYPERTENSION'] = conditions.astype(int)

In [83]:
processed_df['HAS_HYPERTENSION'].value_counts()

HAS_HYPERTENSION
0    7192
1    2430
Name: count, dtype: int64

#### 당뇨여부

> pddm == 2 or druginscu == 2 or glu0 >= 126 or hba1c >= 6.5 or glu120 >= 200

In [84]:
pddm = AS1_06_DISEASE[["AS1_PDDM"]]
druginscu = AS1_08_DRUG[['AS1_DRUGINSCU']]
glu0 = AS1_18_BIOCHEM[['AS1_GLU0_TR']]
glu120 = AS1_18_BIOCHEM[['AS1_GLU120_TR']]
hba1c = AS1_18_BIOCHEM[['AS1_HBA1C']]

df = pd.concat([pddm, druginscu, glu0, glu120, hba1c], axis=1)

In [85]:
conditions = (
    (df['AS1_PDDM'] == 2) | 
    (df['AS1_DRUGINSCU'] == 2) | 
    (df['AS1_GLU0_TR'] >= 126) | 
    (df['AS1_GLU120_TR'] >= 200) |
    (df['AS1_HBA1C'] >= 6.5)
)

processed_df['HAS_DIABETES'] = conditions.astype(int)

In [86]:
processed_df['HAS_DIABETES'].value_counts()

HAS_DIABETES
0    8237
1    1385
Name: count, dtype: int64

#### 고지혈증
> tchl >= 240 or LDL >= 160 or TG >= 200 or HDL < 40

- 현대인은 모두 고지혈증인가? 어떻게 2가 없지

In [87]:
pdlp = AS1_06_DISEASE[["AS1_PDLP"]]
drugslcu = AS1_08_DRUG[['AS1_DRUGSLCU']]
tchl = AS1_18_BIOCHEM[['AS1_TCHL_TR']]
hdl = AS1_18_BIOCHEM[['AS1_HDL_TR']]
tg = AS1_18_BIOCHEM[['AS1_TG_TR']]

cholesterol_df = pd.concat([pdlp, drugslcu, tchl, hdl, tg], axis=1)
cholesterol_df['AS1_LDL_TR'] = cholesterol_df['AS1_TCHL_TR'] - cholesterol_df['AS1_HDL_TR'] - cholesterol_df['AS1_TG_TR'] / 5

In [88]:
cholesterol_df['AS1_TG_TR']/5

DIST_ID
NIH23B1298125     39.6
NIH23B1159376     62.0
NIH23B1785393     66.8
NIH23B1463054     21.0
NIH23B1751168     19.0
NIH23B1703619     11.6
NIH23B1699060     48.0
NIH23B1971042     27.2
NIH23B1338120     60.6
NIH23B1475963     45.4
NIH23B1816289     23.8
NIH23B1498753     20.6
NIH23B1054423     20.6
NIH23B1831908     32.4
NIH23B1743677     41.8
NIH23B1961177     67.4
NIH23B1909485     22.2
NIH23B1758148     22.4
NIH23B1111465     19.0
NIH23B1620823     44.2
NIH23B1110112     23.6
NIH23B1250466     25.4
NIH23B1309271     53.8
NIH23B1479282     18.0
NIH23B1525476     37.4
NIH23B1533757     28.8
NIH23B1433593     17.2
NIH23B1726793     20.2
NIH23B1879696    124.8
NIH23B1997664     27.0
NIH23B1160975     22.0
NIH23B1226704     22.2
NIH23B1815817     16.8
NIH23B1526683     26.6
NIH23B1217067     41.4
NIH23B1714692     26.2
NIH23B1169285     30.6
NIH23B1210354     39.4
NIH23B1932889     18.6
NIH23B1270301     36.8
NIH23B1361302     41.8
NIH23B1020344     45.2
NIH23B1353627     21.8
NIH

In [89]:
cholesterol_df.head()

,AS1_PDLP,AS1_DRUGSLCU,AS1_TCHL_TR,AS1_HDL_TR,AS1_TG_TR,AS1_LDL_TR
DIST_ID,,,,,,
NIH23B1298125,1.0,77777.0,159.0,36.0,198.0,83.4
NIH23B1159376,1.0,NaN,156.0,30.0,310.0,64.0
NIH23B1785393,1.0,77777.0,161.0,44.0,334.0,50.2
NIH23B1463054,1.0,77777.0,161.0,40.0,105.0,100.0
NIH23B1751168,1.0,NaN,151.0,44.0,95.0,88.0


In [90]:
conditions = (
    (cholesterol_df['AS1_PDLP'] == 2) | 
    (cholesterol_df['AS1_DRUGSLCU'] == 2) | 
    (cholesterol_df['AS1_TCHL_TR'] >= 240) | 
    (cholesterol_df['AS1_LDL_TR'] >= 160) |
    (cholesterol_df['AS1_TG_TR'] >= 200) |
    (cholesterol_df['AS1_HDL_TR']) < 40
)

processed_df['HAS_HYPERLIPIDEMI'] = conditions.astype(int)


In [91]:
processed_df['HAS_HYPERLIPIDEMI'].value_counts()

HAS_HYPERLIPIDEMI
1    9622
Name: count, dtype: int64

#### HOMA_IR, 허리둘레, BMI
> HOMA_IR=fasting glucose (mg/dL) X fasting insulin (mU/L) / 405

- AS1_Glu0_TR: mg/dL
- AS1_Ins0: µIU/㎖

- AS1_INS0 µIU/㎖ == uU/mL == mU/L
    - µ == u // 일부 이렇게 표시하는 사람 있음
    - IU == U // 같은 거임
    - 둘은 같다. 근거: https://www.quora.com/Any-one-can-help-how-I-can-convert-between-uU-ml-to-mcunit-ml-for-insulin-measuring-to-use-it-in-HOMA-IR-calculation

In [92]:
# HOMA-IR
processed_df['HOMA_IR'] = (AS1_18_BIOCHEM['AS1_GLU0_TR'] * AS1_18_BIOCHEM['AS1_INS0']) / 405

In [93]:
pd.options.display.max_rows = None
print(len(df[AS1_18_BIOCHEM['AS1_GLU0_TR'] == 99999]))
print(len(df[AS1_18_BIOCHEM['AS1_INS0'] == 99999]))

0
0


In [94]:
# 허리둘레

processed_df['WAIST_AVG'] = AS1_19_ANTHRO[['AS1_WAIST1', 'AS1_WAIST2', 'AS1_WAIST3']].mean(axis=1)

In [95]:
# BMI

processed_df['BMI'] = ((AS1_19_ANTHRO['AS1_HEIGHT'] / 100) ** 2) * AS1_19_ANTHRO['AS1_WEIGHT']

In [96]:
processed_df['HOMA_IR'].describe()

count    9342.000000
mean        1.667727
std         1.264024
min         0.016543
25%         1.046914
50%         1.453827
75%         2.044938
max        53.906667
Name: HOMA_IR, dtype: float64

#### remnant cholesterol
remnant cholesterol = total cholesterol – high-density lipoprotein (HDL) cholesterol – LDL-C

In [97]:
cholesterol_df.head()

,AS1_PDLP,AS1_DRUGSLCU,AS1_TCHL_TR,AS1_HDL_TR,AS1_TG_TR,AS1_LDL_TR
DIST_ID,,,,,,
NIH23B1298125,1.0,77777.0,159.0,36.0,198.0,83.4
NIH23B1159376,1.0,NaN,156.0,30.0,310.0,64.0
NIH23B1785393,1.0,77777.0,161.0,44.0,334.0,50.2
NIH23B1463054,1.0,77777.0,161.0,40.0,105.0,100.0
NIH23B1751168,1.0,NaN,151.0,44.0,95.0,88.0


In [98]:
processed_df['RC'] = cholesterol_df["AS1_TCHL_TR"] - cholesterol_df["AS1_HDL_TR"] - cholesterol_df["AS1_LDL_TR"]

In [99]:
processed_df['RC'].describe()

count    9619.000000
mean       32.428860
std        20.959599
min         6.400000
25%        20.000000
50%        27.000000
75%        38.200000
max       311.400000
Name: RC, dtype: float64

#### 동맥경화지수(Atherogenic Index, AI)

- Fiordaliso의 계산법(Shin et al., 2010)
- AI = ([총콜레스테롤]-[HDL콜레스테롤]/ [HDL콜레스테롤) 

In [100]:
processed_df['AI'] = (cholesterol_df["AS1_TCHL_TR"] - cholesterol_df["AS1_HDL_TR"]) / cholesterol_df["AS1_HDL_TR"]

In [101]:
processed_df['AI'].describe()

count    9620.000000
mean        3.456762
std         1.132368
min        -0.200000
25%         2.631377
50%         3.357143
75%         4.170343
max        10.875000
Name: AI, dtype: float64

## 결과

In [102]:
result_df = pd.concat([result, processed_df], axis=1)

In [103]:
result_df.head()

,AS1_SEX,AS1_AGE,AS1_MARRYA,AS1_JOBB,AS1_EDUA,AS1_INCOME,AS1_DRINK,AS1_SMOKEA,AS1_HVSMAM,AS1_PHYACTL,...,NEAP,DII,HAS_HYPERTENSION,HAS_DIABETES,HAS_HYPERLIPIDEMI,HOMA_IR,WAIST_AVG,BMI,RC,AI
DIST_ID,,,,,,,,,,,,,,,,,,,,,
NIH23B1298125,1,53,2.0,2.0,3.0,5.0,3.0,1.0,0.0,1.0,...,25.068314,2.062933,1,1,1,2.721235,91.933333,200.954880,39.6,3.416667
NIH23B1159376,2,44,2.0,5.0,1.0,2.0,1.0,0.0,0.0,7.0,...,20.677190,2.699810,0,0,1,0.306667,87.066667,135.594778,62.0,4.200000
NIH23B1785393,1,47,2.0,3.0,1.0,2.0,3.0,3.0,8.0,2.0,...,42.987319,1.297841,0,0,1,0.911111,80.333333,170.473543,66.8,2.659091
NIH23B1463054,2,43,2.0,3.0,1.0,2.0,3.0,0.0,0.0,2.0,...,43.921116,1.274731,0,0,1,1.274074,98.500000,170.236012,21.0,3.025000
NIH23B1751168,1,61,2.0,3.0,3.0,1.0,3.0,3.0,20.0,0.0,...,28.310385,1.615857,0,0,1,0.380247,80.666667,159.671232,19.0,2.431818


In [105]:
result_df.to_csv("./Dataset/result.csv")

In [106]:
# 77777
for col in result_df.columns:
    count = (result_df[col] == 77777).sum()
    if count > 0:
        print(f'Column {col} contains {count} instances of 77777')

In [107]:
# NaN
for col in result_df.columns:
    count = result_df[col].isna().sum()
    if count > 0:
        print(f'Column {col} contains {count} instances of NaN')

Column AS1_MARRYA contains 38 instances of NaN
Column AS1_JOBB contains 40 instances of NaN
Column AS1_EDUA contains 53 instances of NaN
Column AS1_INCOME contains 133 instances of NaN
Column AS1_DRINK contains 37 instances of NaN
Column AS1_SMOKEA contains 84 instances of NaN
Column AS1_HVSMAM contains 140 instances of NaN
Column AS1_PHYACTL contains 130 instances of NaN
Column AS1_PHYACTM contains 195 instances of NaN
Column AS1_PHYACTH contains 156 instances of NaN
Column AS1_HEALTH contains 28 instances of NaN
Column AS1_TIED contains 99 instances of NaN
Column AS1_SLPAMSF contains 50 instances of NaN
Column AS1_RGMEALFQA contains 63 instances of NaN
Column HOMA_IR contains 280 instances of NaN
Column WAIST_AVG contains 8 instances of NaN
Column BMI contains 4 instances of NaN
Column RC contains 3 instances of NaN
Column AI contains 2 instances of NaN
